## Installation Reqs (Linux Ubuntu)
1. <code> python3 -m pip install playwright
playwright install </code>

In [2]:
## WEB SCRAPING CELL: 
from playwright.async_api import async_playwright
import asyncio
async def main():
  with async_playwright() as p:
      browser = await p.chromium.launch(
           proxy={
               'server': "http://pr.oxylabs.io:7777",
               "username": "USERNAME",
               "password": "PASSWORD"
               },
           headless=False
       )